In [5]:
import random
import numpy as np
import cv2
import os
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import ToTensor, Normalize, Compose, ToPILImage
from os.path import join
from os import listdir
from torchsummary import summary
import time
import argparse
import models.models.DnCNN as DnCNN, models.models.ResNet as ResNet, models.models.RFDN as RFDN
import models.models.DRLN as DRLN, models.models.pix2pix as pix2pix, models.models.pix2pix2 as pix2pix2
from models.models.kbnet_s_arch import KBNet_s
from models.models.kbnet_l_arch import KBNet_l
from models.models.network_swinir import SwinIR as net
from models.utils import util_calculate_psnr_ssim as util
import models.utils.vgg_loss, models.utils.vgg_perceptual_loss
import matplotlib.pyplot as plt
from models.utils.param import seed_everything

model_list = ['DnCNN', 'ResNet18', 'RFDN', 'DRLN', 'pix2pix', 'pix2pix2', 'swinir']

sel_model = model_list[6]
pth_num =6
noise = False
val = 0.1
show_after_cv = False
clean_nrom = False
if noise: pth = '.pth'
else: pth = '_clean.pth' 
if sel_model=='pix2pix' or sel_model=='pix2pix2': pth='_g.pth'
save_record = False # 기록용 폴더에 있으면
pth_dir ='./save/best_'+sel_model+'_model'+str(pth_num)+pth
if save_record:
    pth_dir ='./기록용/'+sel_model+'/'+'best_'+sel_model+'_model'+str(pth_num)+pth
# load_loss_pth ='./ResNet18_model2.pth'
 # SwinIR.init.pth
pth_dir = './save/SwinIR.32Win.pth'

cal_start = 0
display_number = 1000                         # plt로 디스플레이할 이미지의 개수
residual_output_paths = '/content/residuals'           # 잔차 이미지를 저장할 폴더
noisy_image_paths = '/local_datasets/MLinP/train/scan'
clean_image_paths = '/local_datasets/MLinP/train/clean'

seed_everything(42)
output = False                                         # residual을 파일로 출력할지 여부


def load_img(filepath):
    img = cv2.imread(filepath)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def cvt_to_image(image, norm=False):
    image = image.cpu().squeeze(0)
    image = torch.clamp(image, 0, 1)  # 이미지 값을 0과 1 사이로 클램핑
    if norm:
        image = ToPILImage()(image*0.5+0.5)
    else:
        image = ToPILImage()(image)
    return image

class CustomDatasetTest(data.Dataset):
    def __init__(self, noisy_image_paths, clean_image_paths, transform=None, transform_clean=None):
        self.noisy_image_paths = [join(noisy_image_paths, x) for x in listdir(noisy_image_paths)]
        self.clean_image_paths = [join(clean_image_paths, x) for x in listdir(clean_image_paths)]
        self.transform = transform
        self.transform_clean = transform_clean

    def __len__(self):
        return len(self.noisy_image_paths)

    def __getitem__(self, index):
        
        noisy_image_path = self.noisy_image_paths[index]
        noisy_image = load_img(self.noisy_image_paths[index])
        clean_image_path = self.clean_image_paths[index]
        clean_image = load_img(self.clean_image_paths[index])
        
        if self.transform:
            noisy_image = self.transform(noisy_image)
            clean_image = self.transform_clean(clean_image)

        return noisy_image, noisy_image_path, clean_image, clean_image_path

def tensor_to_yuv(images):
    i =[]
    for image in images:
        image = torch.clamp(image, 0, 1)  # 이미지 값을 0과 1 사이로 클램핑
        image = ToPILImage()(image)
        image = np.array(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
        i.append(image)
    return i

def cal_mae_loss(ps,ts, y=False):
    abss = 0.0
    for p, t in zip(ps,ts):
        if not y:
            p = p[:, :, 0]
            t = t[:, :, 0]
        abss += abs(np.mean(p.flatten()) - np.mean(t.flatten()))
    return abss

def BilateralBlur(image):
    image = image.squeeze(0)
    image = torch.clamp(image,0,1)
    image = ToPILImage()(image)
    image = np.array(image)
    image = cv2.bilateralFilter(image, 3, 10 ,5)
    image = ToTensor()(image)
    image = image.unsqueeze(0)
    return image
    

if sel_model == 'DnCNN':
    model = DnCNN.DnCNN()
elif sel_model == 'ResNet18':
    model = ResNet.ResNet18()
elif sel_model == 'RFDN':
    model = RFDN.RFDN()
elif sel_model == 'DRLN':
    model = DRLN.DRLN()
elif sel_model == 'pix2pix':
    model = pix2pix.Generator()
elif sel_model == 'pix2pix2':
    model = pix2pix2.GeneratorUNet()
    model_D = pix2pix2.Discriminator()
elif sel_model == 'swinir':
    model = net(upscale=1, in_chans=3, img_size=32, window_size=8,
                    img_range=1., depths=[6, 6, 6, 6, 6], embed_dim=180, num_heads=[6, 6, 6, 6, 6],
                    mlp_ratio=2, upsampler='', resi_connection='1conv')

device = torch.device('cuda' if torch.cuda.is_available() else 'mps:0' if torch.backends.mps.is_available() else 'cpu')
# device = torch.device('cpu')
model.load_state_dict(torch.load(pth_dir)) #, map_location='cpu'))
model.eval()
# GPU 사용 여부 확인
model.to(device)

test_transform = Compose([
    # BilateralBlur(512),
    ToTensor(),
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

if clean_nrom:
    test_clean_transform = Compose([
        # BilateralBlur(512),
        ToTensor(),
        Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
else:
    test_clean_transform = Compose([
        # BilateralBlur(512),
        ToTensor()
    ])
# 데이터셋 로드 및 전처리
dataset = CustomDatasetTest(noisy_image_paths, clean_image_paths ,transform=test_transform,transform_clean=test_clean_transform)
# val 분할
train_size = int(len(dataset)*(1-val))
val_size = int(len(dataset)*(val))
train_dataset, val_dataset = random_split(dataset,[train_size,val_size])
# 데이터 로더 설정
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=True)
tot_mae = 0.0
tot_after_mae = 0.0
mae_after_loss = 0.0
iii = 0
print(len(val_dataset),'개 하는중')
with torch.no_grad():
    for noisy_image, _, clean_image, _  in val_loader:
        iii+=1
        if iii <= cal_start:
            continue
        # noisy_image = BilateralBlur(noisy_image)
        noisy_image = noisy_image.to(device)
        if noise:
            pred_noise = model(noisy_image)
            denoised_image = noisy_image - pred_noise  
        else:
            denoised_image = model(noisy_image) #.tolist()
            if show_after_cv:
                denoised_after_image = BilateralBlur(denoised_image)

            pred_noise = (noisy_image.cpu()*0.5+0.5) - denoised_image.cpu()
        # denoised_image를 CPU로 이동하여 이미지 저장
        residual_image = (noisy_image.cpu()*0.5+0.5) - clean_image

        mae_loss = cal_mae_loss(tensor_to_yuv(denoised_image),tensor_to_yuv(clean_image))
        tot_mae += mae_loss
        if show_after_cv:
            mae_after_loss = cal_mae_loss(tensor_to_yuv(denoised_after_image),tensor_to_yuv(clean_image))
            tot_after_mae += mae_after_loss
            denoised_after_image = cvt_to_image(denoised_after_image, clean_nrom)
        
        print('This MAE: ',mae_loss)
        print('This after MAE: ',mae_after_loss)
        if False:
            print(_)
            clean_image = cvt_to_image(clean_image, clean_nrom)
            denoised_image = cvt_to_image(denoised_image, clean_nrom)
            noisy_image = cvt_to_image(noisy_image,norm=True)
            residual_image = cvt_to_image(residual_image, norm=False)
            pred_noise = cvt_to_image(pred_noise, norm=False)
            plt.figure(figsize=(20,13))
            plt.subplot(2,3,1)
            plt.imshow(noisy_image, cmap='gray')
            plt.axis('off')
            plt.subplot(2,3,2)
            plt.imshow(clean_image, cmap='gray')
            plt.axis('off')
            plt.subplot(2,3,3)
            plt.imshow(denoised_image, cmap='gray')
            plt.axis('off')
            plt.subplot(2,3,4)
            plt.imshow(residual_image, cmap='gray')
            plt.axis('off')
            plt.subplot(2,3,5)
            plt.imshow(pred_noise, cmap='gray')
            plt.axis('off')
            if show_after_cv:
                plt.subplot(2,3,6)
                plt.imshow(denoised_after_image, cmap='gray')
                plt.axis('off')
            plt.show()
        if iii == cal_start + display_number:
            break
print((iii-cal_start),'개의', 'total MAE: ', tot_mae/(iii-cal_start))
print((iii-cal_start),'개의', 'total after MAE: ', tot_after_mae/(iii-cal_start))

1000 개 하는중
This MAE:  0.9498977661132812
This after MAE:  0.0
This MAE:  4.4112396240234375
This after MAE:  0.0
This MAE:  4.810291290283203
This after MAE:  0.0
This MAE:  0.2857170104980469
This after MAE:  0.0
This MAE:  0.28414154052734375
This after MAE:  0.0
This MAE:  2.7982635498046875
This after MAE:  0.0
This MAE:  0.5351791381835938
This after MAE:  0.0
This MAE:  4.840045928955078
This after MAE:  0.0
This MAE:  1.5573234558105469
This after MAE:  0.0
This MAE:  3.083820343017578
This after MAE:  0.0
This MAE:  1.091064453125
This after MAE:  0.0
This MAE:  6.671512603759766
This after MAE:  0.0
This MAE:  1.6441001892089844
This after MAE:  0.0
This MAE:  1.0091552734375
This after MAE:  0.0
This MAE:  4.883289337158203
This after MAE:  0.0
This MAE:  1.4093742370605469
This after MAE:  0.0
This MAE:  4.846652984619141
This after MAE:  0.0
This MAE:  3.1518821716308594
This after MAE:  0.0
This MAE:  3.1190757751464844
This after MAE:  0.0
This MAE:  1.2411384582519531
Th

In [15]:
import random
import numpy as np
import cv2
import os
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import ToTensor, Normalize, Compose, ToPILImage
import torchvision.transforms as tf
from os.path import join
from os import listdir
from torchsummary import summary
import time
import argparse
import models.models.DnCNN as DnCNN, models.models.ResNet as ResNet, models.models.RFDN as RFDN
import models.models.DRLN as DRLN, models.models.pix2pix as pix2pix, models.models.pix2pix2 as pix2pix2
from models.models.network_swinir import SwinIR as net
# from models.utils.param import param_check, seed_everything
from models.utils.param import seed_everything
import models.utils.vgg_loss, models.utils.vgg_perceptual_loss
import matplotlib.pyplot as plt

model_list = ['DnCNN', 'ResNet18', 'RFDN', 'DRLN', 'pix2pix']

sel_model = model_list[4]
pth_num =1
noise = True
val = 0.01
if noise: pth = '.pth'
else: pth = '_clean.pth' 
if sel_model=='pix2pix' or sel_model=='pix2pix2': pth='_g.pth'
save_record = False # 기록용 폴더에 있으면
pth_dir ='./save/best_'+sel_model+'_model'+str(pth_num)+pth
if save_record:
    pth_dir ='./기록용/'+sel_model+'/'+'best_'+sel_model+'_model'+str(pth_num)+pth
# load_loss_pth ='./ResNet18_model2.pth'

display_number = 10                        # plt로 디스플레이할 이미지의 개수
residual_output_paths = '/content/residuals'           # 잔차 이미지를 저장할 폴더
noisy_image_paths = '/local_datasets/MLinP/train/scan'
clean_image_paths = '/local_datasets/MLinP/train/clean'

seed_everything(42)
output = False                                         # residual을 파일로 출력할지 여부


def load_img(filepath):
    img = cv2.imread(filepath)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def cvt_to_image(image):
    image = image.cpu().squeeze(0)
     # denoised_image = torch.clamp(denoised_image, 0, 1)  # 이미지 값을 0과 1 사이로 클램핑
    image = ToPILImage()(image*0.5+0.5)
    image = np.array(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
    image = image[:, :, 0]
    return image

class CustomDatasetTest(data.Dataset):
    def __init__(self, noisy_image_paths, clean_image_paths, transform=None):
        self.noisy_image_paths = [join(noisy_image_paths, x) for x in listdir(noisy_image_paths)]
        self.clean_image_paths = [join(clean_image_paths, x) for x in listdir(clean_image_paths)]
        self.transform = transform

    def __len__(self):
        return len(self.noisy_image_paths)

    def __getitem__(self, index):
        
        noisy_image_path = self.noisy_image_paths[index]
        noisy_image = load_img(self.noisy_image_paths[index])
        clean_image_path = self.clean_image_paths[index]
        clean_image = load_img(self.clean_image_paths[index])
        
        if self.transform:
            noisy_image = self.transform(noisy_image)
            clean_image = self.transform(clean_image)

        return noisy_image, noisy_image_path, clean_image, clean_image_path

# Transform Definition
class RGBtoYcrcb(object):
    def __init__(self, ):
        return 

    def __call__(self,sample):
        image = sample
        image = cv2.cvtColor(image,cv2.COLOR_RGB2YCrCb)
        return image

if sel_model == 'DnCNN':
    model = DnCNN.DnCNN()
elif sel_model == 'ResNet18':
    model = ResNet.ResNet18()
elif sel_model == 'RFDN':
    model = RFDN.RFDN()
elif sel_model == 'DRLN':
    model = DRLN.DRLN()
elif sel_model == 'pix2pix':
    model = pix2pix.Generator()
elif sel_model == 'pix2pix2':
    model = pix2pix2.GeneratorUNet()
    model_D = pix2pix2.Discriminator()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'mps:0' if torch.backends.mps.is_available() else 'cpu')
# device = torch.device('cpu')
model.load_state_dict(torch.load(pth_dir, map_location='cpu'))
model.eval()

# GPU 사용 여부 확인
model.to(device)

test_transform = Compose([
    # BilateralBlur(512),
    ToTensor(),
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])
def tensor_to_yuv(image):
    image = torch.clamp(image, 0, 1)  # 이미지 값을 0과 1 사이로 클램핑
    image = ToPILImage()(image)
    image = np.array(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    return

def cal_mae_loss(p,t, y=False):
    if not y:
        p = p[:, :, 0]
        t = t[:, :, 0]
    return abs(np.mean(p.flatten()) - np.mean(t.flatten()))

# 데이터셋 로드 및 전처리
dataset = CustomDatasetTest(noisy_image_paths, clean_image_paths ,transform=test_transform)
# val 분할
train_size = int(len(dataset)*(1-val))
val_size = int(len(dataset)*(val))
train_dataset, val_dataset = random_split(dataset,[train_size,val_size])
# 데이터 로더 설정
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
tot_mae = 0.0






for noisy_image, _, clean_image, _  in val_loader:
    noisy_image = noisy_image.to(device)
    if noise:
        pred_noise = model(noisy_image)
        denoised_image = noisy_image - pred_noise  
    else:
        denoised_image = model(noisy_image)
        pred_noise = noisy_image - denoised_image 
    # denoised_image를 CPU로 이동하여 이미지 저장
    residual_image = noisy_image.cpu() - clean_image
    clean_image = cvt_to_image(clean_image)
    denoised_image = cvt_to_image(denoised_image)
    noisy_image = cvt_to_image(noisy_image)
    residual_image = cvt_to_image(residual_image)
    pred_noise = cvt_to_image(pred_noise)
    mae_loss = cal_mae_loss(denoised_image,clean_image,True)
    tot_mae += mae_loss
    print('This MAE: ',mae_loss)
    if True:
        plt.figure(figsize=(20,13))
        plt.subplot(2,3,1)
        plt.imshow(noisy_image, cmap='gray')
        plt.axis('off')
        plt.subplot(2,3,2)
        plt.imshow(clean_image, cmap='gray')
        plt.axis('off')
        plt.subplot(2,3,3)
        plt.imshow(denoised_image, cmap='gray')
        plt.axis('off')
        plt.subplot(2,3,4)
        plt.imshow(residual_image, cmap='gray')
        plt.axis('off')
        plt.subplot(2,3,5)
        plt.imshow(pred_noise, cmap='gray')
        plt.axis('off')
        plt.show()
print(len(val_loader),'개의', 'total MAE: ', tot_mae/len(val_loader))
# avg_residual = torch.mean(residuals,axis=0)

This MAE:  53.42594909667969
This MAE:  55.935001373291016
This MAE:  21.430633544921875
This MAE:  6.248546600341797
This MAE:  41.061161041259766
This MAE:  86.50758743286133
This MAE:  47.986595153808594
This MAE:  29.028141021728516
This MAE:  83.20027160644531
This MAE:  12.48593521118164
This MAE:  113.15591812133789
This MAE:  96.23281478881836
This MAE:  27.004878997802734
This MAE:  113.36851501464844
This MAE:  48.8288459777832
This MAE:  80.69601821899414
This MAE:  11.784053802490234
This MAE:  15.175090789794922
This MAE:  109.66451644897461
This MAE:  57.6977653503418
This MAE:  103.34006881713867
This MAE:  19.74368667602539
This MAE:  40.421165466308594
This MAE:  102.66105651855469
This MAE:  89.98386764526367
This MAE:  38.45386505126953
This MAE:  26.25112533569336
This MAE:  7.453609466552734
This MAE:  112.30030059814453
This MAE:  42.342201232910156
This MAE:  8.397647857666016
This MAE:  95.58625411987305
This MAE:  52.18144607543945
This MAE:  10.93276596069336


In [14]:
import random
import numpy as np
import cv2
import os
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import ToTensor, Normalize, Compose, ToPILImage
import torchvision.transforms as tf
from os.path import join
from os import listdir
from torchsummary import summary
import time
import argparse
import models.models.DnCNN as DnCNN, models.models.ResNet as ResNet, models.models.RFDN as RFDN
import models.models.DRLN as DRLN, models.models.pix2pix as pix2pix, models.models.pix2pix2 as pix2pix2
# from models.utils.param import param_check, seed_everything
from models.utils.param import seed_everything
import models.utils.vgg_loss, models.utils.vgg_perceptual_loss
import matplotlib.pyplot as plt

model_list = ['DnCNN', 'ResNet18', 'RFDN', 'DRLN', 'pix2pix', 'pix2pix2']

sel_model = model_list[4]
pth_num =2
noise = True
val = 0.001
if noise: pth = '.pth'
else: pth = '_clean.pth' 
if sel_model=='pix2pix' or sel_model=='pix2pix2': pth='_g.pth'
save_record = False # 기록용 폴더에 있으면
pth_dir ='./save/best_'+sel_model+'_model'+str(pth_num)+pth
if save_record:
    pth_dir ='./기록용/'+sel_model+'/'+'best_'+sel_model+'_model'+str(pth_num)+pth
# load_loss_pth ='./ResNet18_model2.pth'

display_number = 10                        # plt로 디스플레이할 이미지의 개수
residual_output_paths = '/content/residuals'           # 잔차 이미지를 저장할 폴더
noisy_image_paths = '/local_datasets/MLinP/train/scan'
clean_image_paths = '/local_datasets/MLinP/train/clean'

seed_everything(42)
output = False                                         # residual을 파일로 출력할지 여부


def load_img(filepath):
    img = cv2.imread(filepath)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def cvt_to_image(image):
    image = image.cpu().squeeze(0)
     # denoised_image = torch.clamp(denoised_image, 0, 1)  # 이미지 값을 0과 1 사이로 클램핑
    image = ToPILImage()(image*0.5+0.5)
    image = np.array(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
    image = image[:, :, 0]
    return image

def tensor_to_yuv(images):
    i =[]
    for image in images:
        image = torch.clamp(image, 0, 1)  # 이미지 값을 0과 1 사이로 클램핑
        image = ToPILImage()(image)
        image = np.array(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
        i.append(image)
    return i

def cal_mae_loss(ps,ts, y=False):
    abss = 0.0
    for p, t in zip(ps,ts):
        if not y:
            p = p[:, :, 0]
            t = t[:, :, 0]
        abss += abs(np.mean(p.flatten()) - np.mean(t.flatten()))
    return abss

# 데이터셋 로드 및 전처리
dataset = CustomDatasetTest(noisy_image_paths, clean_image_paths ,transform=test_transform)
# val 분할
train_size = int(len(dataset)*(1-val))
val_size = int(len(dataset)*(val))
train_dataset, val_dataset = random_split(dataset,[train_size,val_size])
# 데이터 로더 설정
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
tot_mae = 0.0



for noisy_image, _, clean_image, _  in val_loader:
    noisy_image = noisy_image
    if noise:
        pred_noise = noisy_image
        denoised_image = noisy_image - pred_noise  
    else:
        denoised_image = noisy_image
        pred_noise = noisy_image - denoised_image 
    # denoised_image를 CPU로 이동하여 이미지 저장
    residual_image = noisy_image.cpu() - clean_image

    mae_loss = cal_mae_loss(tensor_to_yuv(denoised_image),tensor_to_yuv(clean_image),False)
    tot_mae += mae_loss
    clean_image = cvt_to_image(clean_image)
    denoised_image = cvt_to_image(denoised_image)
    noisy_image = cvt_to_image(noisy_image)
    residual_image = cvt_to_image(residual_image)
    pred_noise = cvt_to_image(pred_noise)
    print('This MAE: ',mae_loss)
    if False:
        plt.figure(figsize=(20,13))
        plt.subplot(2,3,1)
        plt.imshow(noisy_image, cmap='gray')
        plt.axis('off')
        plt.subplot(2,3,2)
        plt.imshow(clean_image, cmap='gray')
        plt.axis('off')
        plt.subplot(2,3,3)
        plt.imshow(denoised_image, cmap='gray')
        plt.axis('off')
        plt.subplot(2,3,4)
        plt.imshow(residual_image, cmap='gray')
        plt.axis('off')
        plt.subplot(2,3,5)
        plt.imshow(pred_noise, cmap='gray')
        plt.axis('off')
        plt.show()
print(len(val_loader),'개의', 'total MAE: ', tot_mae/len(val_loader))
# avg_residual = torch.mean(residuals,axis=0)

This MAE:  199.56296157836914
This MAE:  180.14152145385742
This MAE:  137.51591873168945
This MAE:  218.08250045776367
This MAE:  228.17327499389648
This MAE:  199.27943801879883
This MAE:  238.3889045715332
This MAE:  195.14138793945312
This MAE:  236.4406623840332
This MAE:  199.15451431274414
10 개의 total MAE:  203.18810844421387
